# Bab 13: Memuat dan Melakukan Pra-pemrosesan Data dengan TensorFlow

Bab ini akan membahas secara mendalam bagaimana cara memuat (loading) dan melakukan pra-pemrosesan (preprocessing) data secara efisien menggunakan TensorFlow, khususnya dengan fokus pada `tf.data` API dan format TFRecord. Kita juga akan melihat bagaimana mengintegrasikan langkah-langkah pra-pemrosesan ini langsung ke dalam model Keras kita.

## 1. Pendahuluan

Sistem Machine Learning dan Deep Learning seringkali dilatih dengan dataset yang sangat besar, yang mungkin tidak muat dalam memori RAM. Memuat dan melakukan pra-pemrosesan data secara efisien menjadi kunci performa model. TensorFlow menyediakan `tf.data` API untuk mengatasi tantangan ini. Selain itu, kita akan menjelajahi format TFRecord yang merupakan format biner efisien untuk menyimpan data.

### Mengapa Pra-pemrosesan Efisien Penting?

* **Mengatasi Batasan Memori:** Dataset besar seringkali tidak bisa dimuat sepenuhnya ke dalam RAM. `tf.data` memungkinkan data di-stream dari disk.
* **Performa Pelatihan:** Pra-pemrosesan yang lambat dapat menjadi *bottleneck* pelatihan, membuat GPU/TPU menganggur. Pipeline data yang efisien memastikan data selalu siap saat dibutuhkan.
* **Konsistensi Pra-pemrosesan:** Mengintegrasikan pra-pemrosesan ke dalam model atau pipeline memastikan konsistensi antara pelatihan dan inferensi.

## 2. The Data API (`tf.data`)

`tf.data` API berpusat pada konsep `tf.data.Dataset`, yang merepresentasikan urutan item data. Dataset ini dapat membaca data secara bertahap dari disk dan menerapkan berbagai transformasi.

### 2.1. Membuat Dataset Sederhana

Untuk memulai, mari kita buat dataset sederhana di dalam RAM menggunakan `tf.data.Dataset.from_tensor_slices()`. Ini akan menghasilkan dataset di mana setiap elemen adalah irisan dari tensor input.

In [1]:
import tensorflow as tf
import numpy as np
import os

# Membuat tensor sederhana
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

print(dataset) # Output: <TensorSliceDataset shapes: (), types: tf.int32>

# Melakukan iterasi pada dataset
print("Items dalam dataset:")
for item in dataset:
    print(item)

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>
Items dalam dataset:
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


**Penjelasan Teoritis:**
`tf.data.Dataset.from_tensor_slices(X)` membuat sebuah dataset di mana setiap elemennya adalah "irisan" (slice) dari dimensi pertama `X`. Dalam kasus ini, karena `X` adalah tensor 1D dari 0 sampai 9, dataset akan berisi 10 elemen skalar, yaitu 0, 1, ..., 9.

### 2.2. Chaining Transformations (Rantai Transformasi)

Setelah memiliki dataset, kita dapat menerapkan berbagai transformasi padanya dengan memanggil metode transformasi dataset. Setiap metode ini mengembalikan dataset baru, sehingga Anda dapat merangkai transformasi.

In [2]:
# Versi alternatif: filter elemen individual sebelum di-batch
dataset = tf.data.Dataset.from_tensor_slices(tf.range(10))
dataset = dataset.repeat(3)
dataset = dataset.map(lambda x: x * 2)

# Memfilter elemen individual yang kurang dari 10 (sebelum batching)
dataset = dataset.filter(lambda x: x < 10) # x di sini adalah skalar

# Kemudian baru batch
dataset = dataset.batch(7)

print("\nDataset setelah filter individual dan batch(7):")
for item in dataset.take(3):
    print(item)


Dataset setelah filter individual dan batch(7):
tf.Tensor([0 2 4 6 8 0 2], shape=(7,), dtype=int32)
tf.Tensor([4 6 8 0 2 4 6], shape=(7,), dtype=int32)
tf.Tensor([8], shape=(1,), dtype=int32)


**Penjelasan Teoritis:**
* `repeat(N)`: Mengulang dataset `N` kali. Jika `N` tidak ditentukan, dataset akan diulang selamanya. Ini tidak menyalin data ke memori, melainkan hanya mengubah logika iterasi.
* `batch(batch_size)`: Mengelompokkan elemen-elemen dataset menjadi *mini-batch* dengan ukuran `batch_size`. *Batch* terakhir mungkin lebih kecil jika jumlah total elemen bukan kelipatan dari `batch_size`. Anda bisa menggunakan `drop_remainder=True` untuk memastikan semua *batch* memiliki ukuran yang sama.
* `map(function)`: Menerapkan fungsi transformasi pada setiap elemen dataset. Fungsi ini harus berupa fungsi TensorFlow yang dapat dikonversi ke `tf.function` (lihat Bab 12). Anda dapat menggunakan `num_parallel_calls` untuk memparalelkan eksekusi fungsi ini di beberapa *thread*.
* `filter(predicate)`: Menyaring elemen-elemen dataset berdasarkan fungsi predikat. Hanya elemen yang mengembalikan `True` yang akan dipertahankan.
* `take(N)`: Hanya mengambil `N` elemen pertama dari dataset. Berguna untuk inspeksi atau debugging.

### 2.3. Mengacak Data (Shuffling)

Untuk pelatihan dengan Gradient Descent yang optimal, instance dalam *training set* harus independen dan terdistribusi secara identik (IID). Mengacak (shuffling) data adalah cara sederhana untuk memastikan ini.

In [3]:
# Membuat dataset dari 0 sampai 9, diulang 3 kali
dataset = tf.data.Dataset.range(10).repeat(3)

# Mengacak dataset dengan buffer_size 5 dan seed 42
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)

print("\nDataset setelah shuffle dan batch (dengan random seed):")
for item in dataset:
    print(item)


Dataset setelah shuffle dan batch (dengan random seed):
tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


**Penjelasan Teoritis:**
* `shuffle(buffer_size)`: Membuat *buffer* internal berukuran `buffer_size`. Saat dataset diminta elemen, ia akan mengambil elemen secara acak dari *buffer* dan menggantinya dengan elemen baru dari dataset sumber. Semakin besar `buffer_size`, semakin efektif pengacakannya.
* Untuk dataset yang tidak muat dalam memori, teknik pengacakan yang lebih canggih melibatkan pembagian data ke dalam beberapa file dan membaca file-file tersebut secara acak, kemudian melakukan *interleaving* (membaca baris secara bergantian) dari file-file tersebut.

### 2.4. Interleaving Lines from Multiple Files (Interleaving Baris dari Banyak File)

Ketika data terlalu besar untuk satu file, atau untuk meningkatkan efisiensi pengacakan, Anda dapat membagi data ke dalam beberapa file dan membaca baris-barisnya secara bersamaan.

In [4]:
# Contoh: Jika Anda memiliki file CSV, unduh California Housing Dataset
# dan bagi menjadi beberapa file CSV kecil.
# Ini adalah bagian yang akan Anda jalankan secara manual atau dengan skrip terpisah
# karena melibatkan operasi file sistem dan pengunduhan data.

# Asumsikan Anda memiliki file-file seperti 'my_train_00.csv', 'my_train_01.csv', dll.
# Untuk demonstrasi, kita akan membuat dummy file paths.
train_filepaths = [f"dummy_data/my_train_{i:02d}.csv" for i in range(5)]
# Buat beberapa dummy file agar kode ini bisa dijalankan
os.makedirs("dummy_data", exist_ok=True)
for filepath in train_filepaths:
    with open(filepath, "w") as f:
        f.write("header\n") # Tulis header dummy
        for i in range(10):
            f.write(f"value_{i},value_{i*2}\n")

# Membuat dataset dari daftar jalur file
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

# Menginterleave baris dari 2 file sekaligus (melewatkan header)
n_readers = 2
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers
)

print("\nDataset setelah interleaving (mengambil 5 baris pertama):")
for line in dataset.take(5):
    print(line.numpy())

# Membersihkan dummy files
for filepath in train_filepaths:
    os.remove(filepath)
os.rmdir("dummy_data")


Dataset setelah interleaving (mengambil 5 baris pertama):
b'value_0,value_0'
b'value_0,value_0'
b'value_1,value_2'
b'value_1,value_2'
b'value_2,value_4'


**Penjelasan Teoritis:**
* `tf.data.Dataset.list_files(filepaths)`: Membuat dataset yang berisi jalur (path) ke file-file. Secara *default*, ini akan mengacak urutan jalur file.
* `interleave(map_func, cycle_length, num_parallel_calls)`: Transformasi yang memungkinkan Anda membaca data dari beberapa sumber secara bersamaan dan menginterleave (mencampur) elemen-elemennya.
    * `map_func`: Sebuah fungsi yang menerima `filepath` dan mengembalikan `Dataset` baru (misalnya, `tf.data.TextLineDataset`).
    * `cycle_length`: Berapa banyak dataset sumber yang akan diinterleave secara bersamaan.
    * `num_parallel_calls`: Berapa banyak *thread* CPU yang akan digunakan untuk membaca file secara paralel. `tf.data.experimental.AUTOTUNE` dapat digunakan untuk otomatisasi.

### 2.5. Preprocessing the Data (Pra-pemrosesan Data)

Data mentah, seperti string CSV, perlu diuraikan (parsed) dan dinormalisasi agar dapat digunakan oleh model Machine Learning.

In [5]:
# Untuk demonstrasi, kita akan membuat fungsi preprocess yang menguraikan string byte CSV
# dan menskalakan fitur. Untuk dataset nyata, Anda perlu menghitung mean dan std
# dari training set terlebih dahulu.

# Dummy mean dan std untuk 8 fitur
X_mean = np.array([10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0], dtype=np.float32)
X_std = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0], dtype=np.float32)
n_inputs = 8

def preprocess_csv_line(line):
    # Asumsikan format CSV: feature1,feature2,...,feature8,target
    # Misal record_defaults untuk 8 fitur float dan 1 target float
    # Gunakan tf.constant([]) untuk target agar error jika ada nilai hilang
    # Penjelasan nilai default: Jika kolom tidak ada, gunakan 0.0. Jika kolom target tidak ada, itu akan menjadi kesalahan.
    defs = [0.0] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)

    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])

    # Skalakan fitur
    x_scaled = (x - X_mean) / X_std
    return x_scaled, y

# Contoh penggunaan
dummy_csv_line = b"11.0,22.0,33.0,44.0,55.0,66.0,77.0,88.0,99.0"
x_processed, y_processed = preprocess_csv_line(dummy_csv_line)
print("\nOutput pra-pemrosesan:")
print("Fitur:", x_processed)
print("Label:", y_processed)


Output pra-pemrosesan:
Fitur: tf.Tensor([1. 1. 1. 1. 1. 1. 1. 1.], shape=(8,), dtype=float32)
Label: tf.Tensor([99.], shape=(1,), dtype=float32)


**Penjelasan Teoritis:**
* `tf.io.decode_csv(line, record_defaults)`: Menguraikan satu baris string CSV menjadi daftar tensor. `record_defaults` digunakan untuk menentukan tipe data setiap kolom dan nilai *default* jika ada nilai yang hilang. Jika `tf.constant([])` digunakan sebagai *default*, `decode_csv` akan menimbulkan *error* jika ada nilai yang hilang di kolom tersebut.
* `tf.stack(tensors)`: Menggabungkan daftar tensor menjadi satu tensor baru di sepanjang sumbu baru.

### 2.6. Putting Everything Together (Menggabungkan Semuanya)

Mari kita satukan semua langkah pra-pemrosesan ke dalam satu fungsi pembantu yang akan membuat dan mengembalikan dataset yang efisien.

In [6]:
import tensorflow as tf
import numpy as np
import os

def create_housing_dataset(filepaths, repeat=1, n_readers=5,
                           n_read_threads=None, shuffle_buffer_size=10000,
                           n_parse_threads=5, batch_size=32,
                           n_inputs=8): # n_inputs sekarang digunakan untuk menentukan ukuran mean/std

    # Dummy mean dan std. Buat ini secara dinamis berdasarkan n_inputs
    # Misalnya, kita bisa membuat array dummy dengan ukuran yang benar
    # atau jika Anda memiliki nilai mean/std sebenarnya, potong/sesuaikan ukurannya.
    dummy_mean_values = np.linspace(10.0, 80.0, 8, dtype=np.float32) # Contoh untuk 8 fitur
    dummy_std_values = np.linspace(1.0, 8.0, 8, dtype=np.float32) # Contoh untuk 8 fitur

    # Gunakan slicing untuk mendapatkan jumlah fitur yang sesuai dengan n_inputs
    X_mean = tf.constant(dummy_mean_values[:n_inputs])
    X_std = tf.constant(dummy_std_values[:n_inputs])

    # Fungsi pra-pemrosesan baris CSV lokal untuk fungsi ini
    def preprocess_local(line):
        defs = [0.0] * n_inputs + [tf.constant([], dtype=tf.float32)]
        fields = tf.io.decode_csv(line, record_defaults=defs)

        x = tf.stack(fields[:-1])
        y = tf.stack(fields[-1:])

        # Operasi sekarang akan dilakukan sepenuhnya di TensorFlow dengan dimensi yang sesuai
        x_scaled = (x - X_mean) / X_std
        return x_scaled, y

    # 1. Daftar file
    filepath_dataset = tf.data.Dataset.list_files(filepaths, seed=42)

    # 2. Interleave (baca baris dari banyak file secara paralel)
    dataset = filepath_dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), # Skip header
        cycle_length=n_readers, num_parallel_calls=n_read_threads
    )

    # 3. Pra-pemrosesan setiap baris (menguraikan dan menskalakan)
    dataset = dataset.map(preprocess_local, num_parallel_calls=n_parse_threads)

    # 4. Acak dan Ulangi
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)

    # 5. Batch dan Prefetch
    return dataset.batch(batch_size).prefetch(1)

# --- Bagian untuk mendemonstrasikan perbaikan ---

# Bersihkan dummy files dari percobaan sebelumnya jika ada
if os.path.exists("dummy_data"):
    for file_name in os.listdir("dummy_data"):
        os.remove(os.path.join("dummy_data", file_name))
    os.rmdir("dummy_data")

# Dummy file paths untuk demonstrasi fungsi gabungan
dummy_filepaths = [f"dummy_data/file_{i}.csv" for i in range(3)]
os.makedirs("dummy_data", exist_ok=True)
for fp in dummy_filepaths:
    with open(fp, "w") as f:
        # Menulis 2 fitur dan 1 target, agar sesuai dengan n_inputs=2
        f.write("feature1,feature2,target\n")
        for j in range(5):
            f.write(f"{j+1},{j*2},{j*3}\n")

# Membuat dataset menggunakan fungsi gabungan dengan n_inputs=2
dummy_full_dataset = create_housing_dataset(dummy_filepaths, repeat=1, n_inputs=2)

print("\nOutput dari fungsi create_housing_dataset (mengambil 2 batch):")
for batch_x, batch_y in dummy_full_dataset.take(2):
    print("Batch X (processed):", batch_x)
    print("Batch Y (target):", batch_y)

# Membersihkan dummy files lagi
for fp in dummy_filepaths:
    os.remove(fp)
os.rmdir("dummy_data")


Output dari fungsi create_housing_dataset (mengambil 2 batch):
Batch X (processed): tf.Tensor(
[[ -9. -10.]
 [ -8.  -9.]
 [ -6.  -7.]
 [ -5.  -6.]
 [ -5.  -6.]
 [ -8.  -9.]
 [ -9. -10.]
 [ -7.  -8.]
 [ -9. -10.]
 [ -6.  -7.]
 [ -6.  -7.]
 [ -8.  -9.]
 [ -7.  -8.]
 [ -5.  -6.]
 [ -7.  -8.]], shape=(15, 2), dtype=float32)
Batch Y (target): tf.Tensor(
[[ 0.]
 [ 3.]
 [ 9.]
 [12.]
 [12.]
 [ 3.]
 [ 0.]
 [ 6.]
 [ 0.]
 [ 9.]
 [ 9.]
 [ 3.]
 [ 6.]
 [12.]
 [ 6.]], shape=(15, 1), dtype=float32)


### 2.7. Prefetching

`prefetch(1)` adalah optimasi performa penting. Ia membuat dataset selalu "satu *batch* di depan". Artinya, saat algoritma pelatihan memproses satu *batch*, dataset sudah bekerja secara paralel untuk menyiapkan *batch* berikutnya (misalnya, membaca data dari disk dan memprosesnya). Ini secara dramatis dapat meningkatkan kinerja dengan memastikan GPU/TPU tidak menganggur menunggu data.

In [7]:
# Konsep Prefetching
# CPU membaca dan memproses data sementara GPU melatih model.
# Ini menciptakan pipeline yang mulus.
# Dataset.prefetch(buffer_size)

# Jika dataset muat dalam memori, gunakan cache()
# dataset = dataset.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(1)
# cache() harus dipanggil setelah transformasi yang intensif komputasi,
# tetapi sebelum shuffling dan repeating.

## 3. The TFRecord Format

Format TFRecord adalah format yang disarankan TensorFlow untuk menyimpan sejumlah besar data dan membacanya secara efisien. Ini adalah format biner yang sederhana dan fleksibel.

### 3.1. Membuat File TFRecord

In [8]:
# Membuat file TFRecord sederhana
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"Ini adalah record pertama")
    f.write(b"Dan ini adalah record kedua")

# Membaca file TFRecord
filepaths = ["my_data.tfrecord"]
tfrecord_dataset = tf.data.TFRecordDataset(filepaths)

print("\nIsi file TFRecord:")
for item in tfrecord_dataset:
    print(item)


Isi file TFRecord:
tf.Tensor(b'Ini adalah record pertama', shape=(), dtype=string)
tf.Tensor(b'Dan ini adalah record kedua', shape=(), dtype=string)


**Penjelasan Teoritis:**
* `tf.io.TFRecordWriter`: Digunakan untuk menulis data biner ke file TFRecord. Anda bisa menulis *byte strings* apa pun.
* `tf.data.TFRecordDataset`: Digunakan untuk membaca data dari satu atau lebih file TFRecord. Secara *default*, ini membaca file satu per satu, tetapi Anda dapat menginterleave-nya untuk paralelisme.

### 3.2. Compressed TFRecord Files (File TFRecord Terkompresi)

File TFRecord dapat dikompresi untuk menghemat ruang disk, terutama saat perlu dimuat melalui jaringan.

In [9]:
# Membuat file TFRecord terkompresi (GZIP)
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed_data.tfrecord", options) as f:
    f.write(b"Record terkompresi pertama")
    f.write(b"Record terkompresi kedua")

# Membaca file TFRecord terkompresi
compressed_tfrecord_dataset = tf.data.TFRecordDataset(
    ["my_compressed_data.tfrecord"],
    compression_type="GZIP"
)

print("\nIsi file TFRecord terkompresi:")
for item in compressed_tfrecord_dataset:
    print(item)

# Membersihkan file TFRecord
os.remove("my_data.tfrecord")
os.remove("my_compressed_data.tfrecord")


Isi file TFRecord terkompresi:
tf.Tensor(b'Record terkompresi pertama', shape=(), dtype=string)
tf.Tensor(b'Record terkompresi kedua', shape=(), dtype=string)


**Penjelasan Teoritis:**
* Anda dapat menentukan `compression_type` (misalnya, "GZIP" atau "ZLIB") saat menulis dan membaca file TFRecord.

### 3.3. Protocol Buffers (Protobufs)

Meskipun setiap record TFRecord dapat menggunakan format biner apa pun, biasanya mereka berisi *serialized protocol buffers* (protobufs). Protobufs adalah format biner yang portabel, dapat diperluas, dan efisien.

In [10]:
# Contoh dasar penggunaan Protobuf (membutuhkan instalasi protobuf-compiler dan kompilasi .proto file)
# Biasanya, TensorFlow menyediakan definisi protobuf yang sudah dikompilasi (seperti tf.train.Example).

# Untuk demonstrasi, kita akan langsung menggunakan tf.train.Example
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

# Membuat tf.train.Example
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }
    )
)

# Menulis Example ke file TFRecord
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

print("\nExample Protobuf berhasil dibuat dan disimpan.")


Example Protobuf berhasil dibuat dan disimpan.


**Penjelasan Teoritis:**
* `tf.train.Example`: Protobuf standar TensorFlow untuk merepresentasikan satu instance data. Ini berisi kamus fitur, di mana setiap fitur dapat berupa daftar *byte strings*, *floats*, atau *integers*.
* `SerializeToString()`: Metode pada objek protobuf untuk mengkonversinya menjadi *byte string*.

### 3.4. Loading and Parsing Examples (Memuat dan Menguraikan Example)

In [11]:
# Mendefinisikan deskripsi fitur untuk parsing
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string), # emails adalah fitur panjang variabel
}

# Membaca dan menguraikan file TFRecord
parsed_dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"])

print("\nMenguraikan Example Protobuf:")
for serialized_example in parsed_dataset:
    parsed_example = tf.io.parse_single_example(serialized_example, feature_description)
    print("Parsed Example:", parsed_example)
    print("Emails (sparse):", parsed_example["emails"])
    # Mengkonversi sparse tensor ke dense tensor
    print("Emails (dense):", tf.sparse.to_dense(parsed_example["emails"], default_value=b""))

# Membersihkan file TFRecord
os.remove("my_contacts.tfrecord")


Menguraikan Example Protobuf:
Parsed Example: {'emails': SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64)), 'id': <tf.Tensor: shape=(), dtype=int64, numpy=123>, 'name': <tf.Tensor: shape=(), dtype=string, numpy=b'Alice'>}
Emails (sparse): SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64))
Emails (dense): tf.Tensor([b'a@b.com' b'c@d.com'], shape=(2,), dtype=string)


**Penjelasan Teoritis:**
* `tf.io.FixedLenFeature`: Digunakan untuk fitur dengan panjang tetap (skalar atau tensor dengan bentuk tetap).
* `tf.io.VarLenFeature`: Digunakan untuk fitur dengan panjang variabel, menguraikannya sebagai `tf.SparseTensor`.
* `tf.sparse.to_dense()`: Mengkonversi `tf.SparseTensor` menjadi `tf.Tensor` biasa, dengan mengisi nilai nol pada elemen yang tidak ada.

### 3.5. Handling Lists of Lists Using the SequenceExample Protobuf

Untuk kasus yang lebih kompleks dengan daftar-daftar (misalnya, dokumen teks dengan daftar kalimat, dan setiap kalimat adalah daftar kata), `tf.train.SequenceExample` protobuf lebih cocok.

In [12]:
# Definisi SequenceExample protobuf (disini disederhanakan, Anda tidak perlu mengkompilasinya)
# sequence_example = tf.train.SequenceExample(...)
# Untuk demonstrasi, kita akan melewati pembuatan SequenceExample dan langsung membahas parsingnya.

# Deskripsi fitur konteks (metadata dokumen)
context_feature_descriptions = {
    "author_id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "title": tf.io.FixedLenFeature([], tf.string, default_value=""),
}

# Deskripsi fitur sequence (daftar kalimat, daftar kata)
sequence_feature_descriptions = {
    "sentences": tf.io.VarLenFeature(tf.int64), # Setiap "sentence" adalah daftar word_id
    "words_in_sentences": tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True), # Jika setiap elemen sequence adalah skalar
}

# Asumsikan Anda memiliki serialized_sequence_example
# serialized_sequence_example = b"..." # Ini harus dari SequenceExample yang sebenarnya

# # Ini hanya contoh sintaks parsing, tidak akan dijalankan tanpa serialized_sequence_example yang valid
# # parsed_context, parsed_feature_lists = tf.io.parse_single_sequence_example(
# #     serialized_sequence_example, context_feature_descriptions,
# #     sequence_feature_descriptions)
# # parsed_sentences = tf.RaggedTensor.from_sparse(parsed_feature_lists["sentences"])

**Penjelasan Teoritis:**
* `tf.train.SequenceExample`: Dirancang untuk data urutan, dengan dua bagian utama: `context` (untuk fitur global/metadata) dan `feature_lists` (untuk fitur urutan yang bervariasi panjangnya).
* `tf.io.parse_single_sequence_example()`: Menguraikan satu `SequenceExample`. Mengembalikan tuple yang berisi kamus fitur konteks dan kamus daftar fitur.
* `tf.io.FixedLenSequenceFeature`: Digunakan dalam `sequence_feature_descriptions` untuk fitur urutan di mana setiap elemen urutan memiliki panjang tetap, tetapi panjang total urutannya dapat bervariasi.
* `tf.RaggedTensor`: Penting untuk merepresentasikan data dengan dimensi "compang-camping" (ragged), di mana baris-baris memiliki panjang yang berbeda.

## 4. Preprocessing the Input Features (Pra-pemrosesan Fitur Input)

Pra-pemrosesan fitur dapat dilakukan *offline* (sebelum pelatihan) atau *on-the-fly* (saat pelatihan atau inferensi). Mengintegrasikan lapisan pra-pemrosesan langsung ke dalam model Keras adalah praktik yang baik untuk konsistensi.

### 4.1. Custom Standardization Layer (Lapisan Standardisasi Kustom)

In [13]:
class Standardization(tf.keras.layers.Layer):
    def adapt(self, data_sample):
        # Hitung mean dan std dari sampel data
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)

    def call(self, inputs):
        # Lakukan standardisasi
        return (inputs - self.means_) / (self.stds_ + tf.keras.backend.epsilon())

# Contoh penggunaan
# Asumsikan X_train_sample adalah sampel data training Anda
X_train_sample = np.random.rand(100, 8).astype(np.float32)

std_layer = Standardization()
std_layer.adapt(X_train_sample)

# Menggunakan lapisan dalam model Keras
model = tf.keras.Sequential([
    std_layer,
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1)
])

# # Contoh compile dan fit model (membutuhkan X_train, y_train yang nyata)
# # model.compile(optimizer="adam", loss="mse")
# # model.fit(X_train, y_train, epochs=1)

**Penjelasan Teoritis:**
* Lapisan kustom ini mewarisi dari `tf.keras.layers.Layer`.
* `adapt(data_sample)`: Metode ini dirancang untuk menghitung statistik yang diperlukan dari sampel data (misalnya, *mean* dan *standard deviation* untuk standardisasi). Ini harus dipanggil sebelum lapisan digunakan dalam pelatihan.
* `call(inputs)`: Metode ini mendefinisikan operasi *forward pass* lapisan.

### 4.2. Encoding Categorical Features Using One-Hot Vectors (Encoding Fitur Kategorikal Menggunakan One-Hot Vectors)

Fitur kategorikal (misalnya, `ocean_proximity`) perlu diubah menjadi representasi numerik. *One-hot encoding* adalah pilihan yang baik untuk kategori dalam jumlah kecil.

In [14]:
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
# Buat indeks untuk setiap kategori
indices = tf.range(len(vocab), dtype=tf.int64)

# Inisialisasi lookup table
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2 # Jumlah bucket untuk Out-Of-Vocabulary (Oov)
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

# Contoh encoding
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
print("\nIndeks kategori:", cat_indices)

# Melakukan one-hot encoding
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)
print("One-hot encoded:", cat_one_hot)


Indeks kategori: tf.Tensor([3 5 1 1], shape=(4,), dtype=int64)
One-hot encoded: tf.Tensor(
[[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(4, 7), dtype=float32)


**Penjelasan Teoritis:**
* `tf.lookup.StaticVocabularyTable`: Membuat tabel pencarian (lookup table) yang memetakan string ke integer.
* `num_oov_buckets`: Menentukan berapa banyak "bucket" yang akan digunakan untuk kategori yang tidak ada dalam kosakata yang ditentukan. Kategori OOV akan di-hash dan dipetakan ke salah satu bucket ini.
* `tf.one_hot(indices, depth)`: Melakukan *one-hot encoding* pada tensor indeks.

### 4.3. Encoding Categorical Features Using Embeddings (Encoding Fitur Kategorikal Menggunakan Embeddings)

Untuk fitur kategorikal dengan jumlah kategori yang besar, *embeddings* adalah pilihan yang lebih efisien daripada *one-hot encoding*. *Embeddings* adalah vektor padat yang dapat dilatih yang merepresentasikan setiap kategori.

In [15]:
import tensorflow as tf
import numpy as np

# Pastikan Anda memiliki 'vocab' dan 'table' yang sudah didefinisikan dari sel sebelumnya.
# Untuk demo ini, saya akan mendefinisikannya kembali secara minimal.
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

embedding_dim = 2 # Dimensi embedding

# Menggabungkan fitur numerik dan embedding dalam model
# Asumsikan regular_inputs adalah tensor fitur numerik
regular_inputs = tf.keras.layers.Input(shape=[8], name="regular_input")
categories_input = tf.keras.layers.Input(shape=[], dtype=tf.string, name="categories_input")

# Mengubah kategori menjadi indeks menggunakan tabel pencarian
# Perbaikan: Tambahkan `output_shape`
cat_indices_layer = tf.keras.layers.Lambda(
    lambda cats: table.lookup(cats),
    output_shape=lambda input_shape: input_shape # Output shape sama dengan input shape
)(categories_input)

# Melakukan embedding
cat_embed_layer = tf.keras.layers.Embedding(
    input_dim=len(vocab) + num_oov_buckets, output_dim=embedding_dim
)(cat_indices_layer)

# Menggabungkan fitur reguler dan embedding
encoded_inputs = tf.keras.layers.concatenate([regular_inputs, cat_embed_layer])

# Model Dense sederhana setelah embedding
outputs = tf.keras.layers.Dense(1)(encoded_inputs)

model_with_embedding = tf.keras.Model(inputs=[regular_inputs, categories_input],
                                       outputs=[outputs])

# Contoh compile dan fit
# model_with_embedding.compile(optimizer="adam", loss="mse")
# dummy_X_num = np.random.rand(10, 8).astype(np.float32)
# dummy_X_cat = np.array(["INLAND"]*5 + ["NEAR BAY"]*5)
# dummy_Y = np.random.rand(10, 1).astype(np.float32)
# model_with_embedding.fit({"regular_input": dummy_X_num, "categories_input": dummy_X_cat}, dummy_Y, epochs=1)

model_with_embedding.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ categories_input    │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None)            │          0 │ categories_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ regular_input       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 2)         │         14 │ lambda[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 10)        │          0 │ regular_input[0]… │
│ (Concatenate)       │                   │            │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         11 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 25 (100.00 B)

 Trainable params: 25 (100.00 B)

 Non-trainable params: 0 (0.00 B)

**Penjelasan Teoritis:**
* `tf.keras.layers.Embedding`: Lapisan ini mengambil indeks integer sebagai input dan mengembalikan vektor *embedding* yang sesuai. Matriks *embedding* itu sendiri adalah bobot yang dapat dilatih oleh model.
* *Embeddings* memungkinkan model mempelajari representasi yang bermakna untuk kategori, di mana kategori yang serupa akan memiliki *embedding* yang dekat dalam ruang *embedding*.

### 4.4. Keras Preprocessing Layers (Lapisan Pra-pemrosesan Keras)

TensorFlow sedang mengembangkan serangkaian lapisan pra-pemrosesan Keras standar (`tf.keras.layers.experimental.preprocessing` atau nanti langsung di `tf.keras.layers`). Ini menyederhanakan pra-pemrosesan data dalam model.

In [16]:
# Contoh Lapisan Pra-pemrosesan Keras (fitur eksperimental, mungkin berbeda di versi TF yang lebih baru)
# Asumsikan Anda memiliki data mentah
# raw_numerical_data = np.random.rand(100, 5).astype(np.float32)
# raw_categorical_data = np.array(["A", "B", "C", "A", "B"]*20)

# Lapisan Normalization (standardisasi)
# normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
# normalizer.adapt(raw_numerical_data) # Hitung mean dan variance
# normalized_data = normalizer(raw_numerical_data)

# Lapisan TextVectorization (untuk teks, atau kategorikal string)
# text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
#     max_tokens=1000, output_mode="int") # max_tokens = ukuran kosakata
# text_vectorizer.adapt(raw_categorical_data) # Bangun kosakata
# encoded_text = text_vectorizer(raw_categorical_data)

# Lapisan Discretization
# discretizer = tf.keras.layers.experimental.preprocessing.Discretization(
#     bin_boundaries=[0.1, 0.5, 0.9]) # Batas untuk membagi data kontinu ke dalam bin
# discretized_data = discretizer(raw_numerical_data[:, 0])

print("\nLapisan Pra-pemrosesan Keras (konsep):")
print("Lapisan ini akan memudahkan pra-pemrosesan data mentah langsung dalam model.")


Lapisan Pra-pemrosesan Keras (konsep):
Lapisan ini akan memudahkan pra-pemrosesan data mentah langsung dalam model.


**Penjelasan Teoritis:**
* Lapisan-lapisan ini (`Normalization`, `TextVectorization`, `Discretization`, dll.) dirancang untuk diintegrasikan langsung ke dalam model Keras.
* Metode `adapt()` pada lapisan-lapisan ini memungkinkan mereka untuk menganalisis sampel data (biasanya *training set*) dan menghitung statistik yang diperlukan (misalnya, *mean*/*std* untuk normalisasi, *vocabulary* untuk teks).
* Setelah di-`adapt`, lapisan dapat digunakan dalam model dan akan menerapkan transformasi yang sama secara konsisten selama pelatihan dan inferensi.

## 5. TF Transform (`tf.Transform`)

`tf.Transform` adalah bagian dari TensorFlow Extended (TFX) dan memungkinkan Anda menulis fungsi pra-pemrosesan tunggal yang dapat dijalankan dalam mode *batch* di seluruh *training set* sebelum pelatihan, dan kemudian diekspor ke `tf.function` untuk digunakan dalam model yang di-*deploy*. Ini mengatasi masalah *training/serving skew*.

In [17]:
# Konsep tf.Transform (membutuhkan instalasi tensorflow_transform dan Apache Beam)
!pip install tensorflow_transform
import tensorflow_transform as tft

# Contoh fungsi pra-pemrosesan (hanya untuk ilustrasi)
def preprocess_with_tft(inputs):
    # inputs adalah sebuah batch dari fitur input
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]

    # Skala fitur numerik
    standardized_age = tft.scale_to_z_score(median_age)

    # Bangun kosakata untuk fitur kategorikal
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)

    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_proximity_id
    }

print("\nKonsep tf.Transform:")
print("Memungkinkan pra-pemrosesan konsisten antara pelatihan dan serving.")


Konsep tf.Transform:
Memungkinkan pra-pemrosesan konsisten antara pelatihan dan serving.


**Penjelasan Teoritis:**
* **Offline Pra-pemrosesan:** `tf.Transform` menjalankan fungsi pra-pemrosesan di seluruh *training set* (*offline*) menggunakan kerangka pemrosesan data terdistribusi (seperti Apache Beam). Selama proses ini, ia menghitung statistik (misalnya, *mean*, *std*, *vocabulary*) yang diperlukan untuk transformasi.
* **Export ke `tf.function`:** Setelah statistik dihitung, `tf.Transform` menghasilkan `tf.function` yang merangkum logika pra-pemrosesan dan nilai-nilai statistik yang dihitung.
* **Konsistensi:** Fungsi `tf.function` ini kemudian dapat menjadi bagian dari model yang diekspor, memastikan bahwa pra-pemrosesan selama *serving* (inferensi) persis sama dengan yang dilakukan selama pelatihan.

## 6. The TensorFlow Datasets (TFDS) Project

TFDS menyediakan cara mudah untuk mengunduh dan memuat banyak dataset umum secara langsung ke dalam format `tf.data.Dataset`.

In [19]:
import tensorflow_datasets as tfds

# Mengunduh dan memuat dataset MNIST
# as_supervised=True akan mengembalikan (features, label) tuples, cocok untuk Keras
dataset, info = tfds.load(name="mnist", batch_size=32, as_supervised=True, with_info=True)

mnist_train, mnist_test = dataset["train"], dataset["test"]

print("\nDataset MNIST dari TFDS:")
print("Info Dataset:", info)
print("Ukuran Training Set:", info.splits["train"].num_examples)

# Contoh iterasi pada dataset
print("\nContoh batch dari MNIST training set:")
for images, labels in mnist_train.take(1):
    print("Shape Gambar:", images.shape)
    print("Shape Label:", labels.shape)
    print("Contoh Label:", labels.numpy())

# Contoh penggunaan langsung dengan Keras
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(10, activation="softmax")
])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(mnist_train, epochs=1)


Dataset MNIST dari TFDS:
Info Dataset: tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/root/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/e

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7735 - loss: 17.1587


**Penjelasan Teoritis:**
* `tfds.load(name, as_supervised, with_info)`: Fungsi utama untuk memuat dataset dari katalog TFDS.
    * `name`: Nama dataset (misalnya, "mnist", "imdb_reviews").
    * `as_supervised=True`: Mengembalikan elemen-elemen sebagai tuple `(features, label)`, ideal untuk model Keras yang dilatih secara *supervised*.
    * `with_info=True`: Mengembalikan metadata tentang dataset.
* TFDS secara otomatis menangani pengunduhan, ekstraksi, dan pembagian dataset menjadi `tf.data.Dataset` objek, yang kemudian dapat digunakan langsung dalam pelatihan model Keras.